## Taxi-V3 using SARSA made by Andrea Bolla - 4482930

SARSA algorithm is a slight variation of the popular Q-Learning algorithm. For a learning agent in any Reinforcement Learning algorithm it’s policy can be of two types:

    On Policy: the learning agent learns the value function according to the current action derived from the policy currently being used. 
    Off Policy: the learning agent learns the value function according to the action derived from another policy. 

Q-Learning technique is an Off Policy technique and uses the greedy approach to learn the Q-value. SARSA technique, on the other hand, is an On Policy and uses the action performed by the current policy to learn the Q-value.

The equation for SARSA depends on the current state, current action, reward obtained, next state and next action. SARSA stands for State Action Reward State Action which symbolizes the tuple (s, a, r, s’, a’).

# Step 1: Importing the required libraries

In [1]:
import sys
sys.path.append('/Users/boez/opt/anaconda3/lib/python3.9/site-packages')

import gym
import numpy as np
import random

sys: is used for the path of my libraries (NB change it with yours)

gym: is a standard API for reinforcement learning, and has diverse collection of reference environments

numpy: is a Python library used for working with arrays

random:  is a module to implement pseudo-random number generators

# Step 2: Building the environment

In [2]:
env = gym.make("Taxi-v3", render_mode="human").env

env.reset()
env.render()

print("Action Space {}".format(env.action_space))

print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


env.reset: Resets the environment and returns a random initial state.

env.step(action): Step the environment by one timestep. Returns:
    
    observation: Observations of the environment

    reward: If your action was beneficial or not

    done: Indicates if we have successfully picked up and dropped off a passenger, also called one episode

    truncated: if episode truncates due to a time limit or a reason that is not defined as part of the task MDP.

    info: Additional info such as performance and latency for debugging purposes

env.render: Renders one frame of the environment from 0-5 where:

    0 = south
    1 = north
    2 = east
    3 = west
    4 = pickup
    5 = dropoff

The 500 states correspond to a encoding of the taxi's location, the passenger's location, and the destination location

In [8]:
env.P[123]

{0: [(1.0, 223, -1, False)],
 1: [(1.0, 23, -1, False)],
 2: [(1.0, 123, -1, False)],
 3: [(1.0, 103, -1, False)],
 4: [(1.0, 123, -10, False)],
 5: [(1.0, 123, -10, False)]}

The 0-5 corresponds to the actions (south, north, east, west, pickup, dropoff) the taxi can perform at our current state in the illustration.

In this env, probability is always 1.0.

The nextstate is the state we would be in if we take the action at this index of the dict

All the movement actions have a -1 reward 
the pickup/dropoff actions have -10 reward in this particular state. 
If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action 

done is used to tell us when we have successfully dropped off a passenger in the right location. 

Each successfull dropoff is the end of an episode

# Step 3: Initializing different parameters

In [4]:
#Defining the different parameters
epsilon = 1 # Total exploration and no exploitation
alpha = 0.3
gamma = 0.95

# Training parameters
n_episodes = 100000  # number of episodes to use for training
n_max_steps = 100   # maximum number of steps per episode

#Initializing the Q-table 500x6
Q = np.zeros((env.observation_space.n, env.action_space.n))

The equation for SARSA depends on the current state, current action, reward obtained, next state and next action.

Q(s_t,a_t) = Q(s_t,a_t) + alpha* ( r_(t+1) + gamma* Q(s_(t+1),a_(t+1)) - Q(s_t,a_t) )

    
Where: 
     
    ϵ (epsilon) is the paramenter which choose between exploration (choosing a random action) and exploitation (choosing actions based on already learned Q-values). 
    
    α (alpha) is the learning rate, it is the extent to which our Q-values are being updated in every iteration.
    
    γ (gamma) is the discount factor  determines how much importance we want to give to future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, insted a discount factor of 0 makes our agent consider only immediate reward (greedy).

# Step 4: Defining utility functions to be used in the learning process

In [5]:
# Function to choose the next action
def choose_action(state):

    action=0

    if np.random.uniform(0, 1) < epsilon:

        action = env.action_space.sample()   # explore

    else:
        
        action = np.argmax(Q[state, :])      # exploit
        
    return action

# Function to update the Q-value
def update(state, state2, reward, action, action2):
    
    predict = Q[state, action]
    target = reward + gamma * Q[state2, action2]
    
    Q[state, action] = Q[state, action] + alpha * (target - predict)

Choose_action() allow the agent to choose the next action, it all depens on the random number and the epsilon value, if epsilon is bigger than the random number the agent will choose to explore, otherwise to exploit and choose in the update Q table the best action.

Update() is used to update the Q table, following the SARSA equation, when the agent choose to explore.

# Step 5: Training the learning agent

To train the agent we use the function see in the step 4. 

We start having a pure exploration to fill the Q table but with the episodes increse we want that the explotation increse too. To obtain that we are decresing epsilon every episode. After 200 episode (20000 step) we will be in a pure explotation state.


Each episode require a different number of steps to complete the task, so we don’t always receive the same reward (each move is minus 1 point).

The minimum reward will be 3 (20 - 17), it takes 17 steps (16 moves + 1 pick up action) if we initialised the taxi and passenger at opposite corners with a drop-off location being the same corner as the taxi’s original position. 

The maximum reward is 15, we cannot expect any higher, if we take the two closest colour squares (Red and Yellow), the agent would use 5 moves (1 to pick up + 4 to move).

# Result

The program should run for hundreds/thousands of episodes (many hours) to learn and have good result, so I will attach some results.

At the beginning we have pure exploration, the agent will explore every situation making a lot of mistakes. We can see that in the firts 100 episode it reached the goal only 3 times.

    Episode:  1
    Goal reached:  0
    Score:  -334

    Episode:  100
    Goal reached:  3
    Score:  -307

After 100 episodes we are in a 50/50 between exploration and explotation, due to the decreasing value of epsilon. Here we can see that the agent is still making mistakes but it start to reach some goals.

    Episode:  201
    Goal reached:  26
    Score:  -100

Now, after 200 episodes we are in pure explotation, the agent should have learned enough to take good decision, we see that in the last 100 episode (201 to 302) it reached the goal 74 times but it's not perfect yet.

    Episode:  302
    Goal reached:  100
    Score:  -13

Then, after 400 episodes the agent knows perfectly how to interact in every situation taking the best action and the shortest way to reach the goal. It's rare that he doesn't reach the goal. 

    Episode:  403
    Goal reached:  193
    Score:  8
    
    Episode:  3475
    Goal reached:  3261
    Score:  8